In [3]:
import pandas as pd
import sqlite3

## create a connection to the database using the library sqlite3

In [57]:
db_connection = sqlite3.connect('../data/checking-logs.sqlite')

## create the new dataframe datamart by joining the tables pageviews and checker using only one query
- the table should have the following columns: uid, labname, checker_ts, first_view_ts
- checker_ts is just a new name of the column timestamp from the checker table, it shows the first commit of the particular lab of the particular user
- first_view_ts is the first visit of a user in the table pageviews
- status = 'ready' still should be a filter
- numTrials = 1 still should be a filter
- labnames still should be from the list: 'laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1', 
- the table should contain only the users and not the admins
- checker_ts and first_view_ts should be parsed as datetime64[ns]

In [69]:
pd.io.sql.read_sql('select * from checker limit 10', db_connection)

,index,status,success,timestamp,numTrials,labname,uid
0,0,checking,0,2020-04-16 21:12:50.740474,5,None,admin_1
1,1,ready,0,2020-04-16 21:12:54.708365,5,code_rvw,admin_1
2,2,checking,0,2020-04-16 21:46:47.769088,7,None,admin_1
3,3,ready,0,2020-04-16 21:46:48.121217,7,lab02,admin_1
4,4,checking,0,2020-04-16 21:53:01.862637,6,code_rvw,admin_1
5,5,ready,0,2020-04-16 21:53:05.373389,6,code_rvw,admin_1
6,6,checking,0,2020-04-17 05:18:51.965864,1,None,None
7,7,ready,0,2020-04-17 05:19:02.744528,1,project1,user_4
8,8,checking,0,2020-04-17 05:22:35.249331,2,project1,user_4
9,9,ready,1,2020-04-17 05:22:45.549397,2,project1,user_4


In [70]:
query = '''
select
    checker.uid as uid,
    labname,
    min(checker.timestamp) as checker_ts,
    min(pageviews.datetime) as first_view_ts
from checker
left join pageviews
    on checker.uid = pageviews.uid
where
    labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1') and
    checker.uid not like "admin%" and
    status = 'ready' and
    numTrials = 1
group by checker.uid, labname
'''

In [71]:
datamart = pd.io.sql.read_sql(query, db_connection, parse_dates=['checker_ts', 'first_view_ts'])

In [72]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   uid            140 non-null    object        
 1   labname        140 non-null    object        
 2   checker_ts     140 non-null    datetime64[ns]
 3   first_view_ts  59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


In [73]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 0 to 115
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   uid            59 non-null     object        
 1   labname        59 non-null     object        
 2   checker_ts     59 non-null     datetime64[ns]
 3   first_view_ts  59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


## using Pandas methods create two dataframes: test and control
- test should have the users that have the values in first_view_ts
- control should have the users that have missing values in first_view_ts
- replace missing values in the control by the average of first_view_ts of the test users, we will use this value for the future analysis
- save both tables into the database, you will use them in the next exercises


In [74]:
test = datamart.dropna()

In [75]:
control = datamart[datamart['first_view_ts'].isna()]

In [76]:
control = control.fillna(test['first_view_ts'].mean())

In [79]:
pd.io.sql.to_sql(test, 'test', db_connection, if_exists='replace')

In [80]:
pd.io.sql.to_sql(control, 'control', db_connection, if_exists='replace')

## close the connection

In [81]:
db_connection.close()